<a href="https://colab.research.google.com/github/lizhieffe/llm_knowledge/blob/main/examples/colab/vLLM_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab was created at 08/17/2025. The workaround may not be needed with fixes from vLLM etc. Pay attention to it.

References
- https://colab.sandbox.google.com/github/noamgat/lm-format-enforcer/blob/main/samples/colab_vllm_integration.ipynb

In [1]:
# @title Install vLLM and dependencies

%%capture

# The 0.10 version of vLLM doesn't load model successfully. So we use 0.9.2 version

# Similar bugs
#   https://github.com/vllm-project/vllm/issues/17618
!pip install vllm==0.9.2 lm-format-enforcer pandas

# This is needed to be compatible with vLLM 0.9.2.
#
# For issue: https://github.com/vllm-project/vllm/issues/17618
!pip install "transformers<4.54.0"

In [2]:
!pip show vllm

Name: vllm
Version: 0.9.2
Summary: A high-throughput and memory-efficient inference and serving engine for LLMs
Home-page: https://github.com/vllm-project/vllm
Author: vLLM Team
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, blake3, cachetools, cloudpickle, compressed-tensors, depyf, einops, fastapi, filelock, gguf, huggingface-hub, lark, llguidance, lm-format-enforcer, mistral_common, msgspec, ninja, numba, numpy, openai, opencv-python-headless, outlines, partial-json-parser, pillow, prometheus-fastapi-instrumentator, prometheus_client, protobuf, psutil, py-cpuinfo, pybase64, pydantic, python-json-logger, pyyaml, pyzmq, ray, regex, requests, scipy, sentencepiece, tiktoken, tokenizers, torch, torchaudio, torchvision, tqdm, transformers, typing_extensions, watchfiles, xformers, xgrammar
Required-by: 


In [3]:
# vLLM uses larger space of VRAM.
# https://www.reddit.com/r/LocalLLaMA/comments/1l8t8n8/huge_vram_usage_with_vllm/
#
# Here we set max context lenth (input + output)
#
# See more on how to optimize: https://qwen.readthedocs.io/en/latest/deployment/vllm.html#faq

import vllm

# llm = vllm.LLM(model="Qwen/Qwen3-0.6B") # OK
# llm = vllm.LLM(model="Qwen/Qwen3-4B") # OK
# llm = vllm.LLM(model="Qwen/Qwen3-8B", max_model_len=4096) # OK on A100. OK on L4 after max_model_len=4096.
# llm = vllm.LLM(model="Qwen/Qwen3-14B", max_model_len=2048) # OK on A100. Not working on L4 after max_model_len=2048, 3072 or 4096
# llm = vllm.LLM(model="Qwen/Qwen3-32B", max_model_len=3072) # Not working on A100 after max_model_len=3072, 4096
# llm = vllm.LLM(model="Qwen/Qwen3-Coder-30B-A3B-Instruct-FP8", max_model_len=4096) # OK on A100 for 3072; hasn't try 4096. For L4, 4096 doesn't work.
llm = vllm.LLM(model="cpatonn/Qwen3-Coder-30B-A3B-Instruct-AWQ-4bit", max_model_len=4096) # OK on L4.

INFO 08-19 02:59:50 [__init__.py:244] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

INFO 08-19 03:00:08 [config.py:841] This model supports multiple tasks: {'generate', 'embed', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 08-19 03:00:08 [config.py:1472] Using max model len 4096
INFO 08-19 03:00:08 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

WARNING 08-19 03:00:18 [__init__.py:2662] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized


In [6]:
# @title Inference example 1 - inference raw string

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = vllm.SamplingParams(temperature=0.8, top_p=0.95, max_tokens=512)

outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: 'Hello, my name is', Generated text: " Maria. I am 25 years old. I was born in New York City, but I grew up in San Francisco. I have two brothers and one sister. My parents are both teachers. I am a college graduate. My favorite subject in school was always English. I have been learning English for more than 20 years. I have a large vocabulary and I am good at writing. I have been working as an English teacher for more than 10 years. I enjoy teaching very much and I love my students. I think I am a good teacher.\n\nWhat is the speaker's favorite subject in school?\nThe speaker's favorite subject in school was English.\nWhat is the speaker's major in college?\nThe speaker's major in college was English.\nWhat is the speaker's job?\nThe speaker's job is an English teacher.\nWhat is the speaker's age?\nThe speaker is 25 years old.\nWhat is the speaker's name?\nThe speaker's name is Maria.\nWhere was the speaker born?\nThe speaker was born in New York City.\nWhere did the speaker g

In [7]:
# @title Inference example 2 - inference structured messages

# 1. Define your system instruction and user prompts
system_prompt = "You are a helpful and concise assistant."
user_prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

formatted_prompts = []

tokenizer = llm.get_tokenizer()

for prompt in user_prompts:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=True)
    formatted_prompts.append(formatted_prompt)

sampling_params = vllm.SamplingParams(temperature=0.8, top_p=0.95, max_tokens=2048)

outputs = llm.generate(formatted_prompts * 8, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Formatted Prompt:\n{prompt!r}\n\nGenerated text:\n{generated_text!r}")
    print("---")

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Formatted Prompt:
'<|im_start|>system\nYou are a helpful and concise assistant.<|im_end|>\n<|im_start|>user\nHello, my name is<|im_end|>\n<|im_start|>assistant\n'

Generated text:
"Hello! It looks like your message was cut off. My name is Qwen, but I'd be happy to help you with whatever you'd like to discuss or ask. How can I assist you today?"
---
Formatted Prompt:
'<|im_start|>system\nYou are a helpful and concise assistant.<|im_end|>\n<|im_start|>user\nThe president of the United States is<|im_end|>\n<|im_start|>assistant\n'

Generated text:
"The President of the United States is the head of state and head of government of the United States. As of my last update in April 2024, Joe Biden is serving as the 46th President of the United States, having taken office on January 20, 2021.\n\nThe President serves a four-year term and is elected through the Electoral College system, with the Vice President serving as President of the Senate. The officeholder is responsible for executing feder